https://docs.python.org/3.4/library/multiprocessing.html

In [20]:
import sys
print(sys.version)
import pandas
print('pandas',pandas.__version__)
import numpy
print('numpy',numpy.__version__)
from multiprocessing import Process
import os
import time

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
pandas 0.23.4
numpy 1.13.3


# introduction to concurrent processes

In [21]:
def proc_info(some_str):
    print('arg =',some_str) # string that was passed in
    print('\nmodule name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def myfunc(a_name):
    """ this function calls another function """
    print('now in myfunc')
    print('process ID =',os.getpid())
    proc_info(a_name)
    print('end of myfunc')

before using multiprocessing, inspect the processes of a normal python execution

In [22]:
print('process ID =',os.getpid(),'\n')
myfunc('mary')

process ID = 211 

now in myfunc
process ID = 211
arg = mary

module name: __main__
parent process: 6
process id: 211
end of myfunc


# launch a separate (serial) process<BR> using multiprocessing module

In [23]:
if __name__ == '__main__':
    proc_info('main code') # display info about this process
    print('\ncreate process')
    p = Process(target=myfunc, args=('bob',)) # from main, call a function
    print('\nstart process')
    p.start()
    print('\njoin')

    p.join()

arg = main code

module name: __main__
parent process: 6
process id: 211

create process

start process
now in myfunc
process ID = 298
arg = bob

module name: __main__
parent process: 211
process id: 298
end of myfunc

join


# multiprocessing for concurrent operations
Now that we've explored having multiple processes, use the processes concurrently

https://docs.python.org/3.4/library/multiprocessing.html#using-a-pool-of-workers

In [24]:
from multiprocessing import Pool

## _example_: map function to list

In [29]:
def sq_fun(x):
    return x*x

In [30]:
%%time

if __name__ == '__main__': # only the parent thread runs the pool
    
    with Pool(5) as p:
        print(p.map(sq_fun, [1, 2, 3]))

[1, 4, 9]
CPU times: user 70 ms, sys: 160 ms, total: 230 ms
Wall time: 343 ms


## _example_: dataframe operation

Create random dataframe

https://stackoverflow.com/questions/32752292/how-to-create-a-data-frame-of-random-integers-with-pandas

In [7]:
df = pandas.DataFrame(
    numpy.random.randint(0,100,size=(100, 4)), 
    columns=list('ABCD')
    )

In [8]:
print('df shape:',df.shape)
df.head()

df shape: (100, 4)


A   B   C   D
0  64  64  34   1
1  64  97  72  61
2   9  29  71  10
3  57  46  66  59
4  10  25  10  28

http://www.racketracer.com/2016/07/06/pandas-in-parallel/

In [9]:
num_partitions = 7 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

In [10]:
def parallelize_dataframe(df, func,num_cores):
    """
    Apply a function to chunks of a dataframe concurrently
    """
    df_split = numpy.array_split(df, num_partitions)
    with Pool(num_cores) as pool:
        df = pandas.concat(pool.map(func, df_split))
    return df

In [11]:
def multiply_columns(data):
    data['K'] = data['A'].apply(lambda x: x*2)
    return data

In [12]:
%time df_modified = parallelize_dataframe(df, multiply_columns,num_cores)

CPU times: user 150 ms, sys: 130 ms, total: 280 ms
Wall time: 406 ms


In [13]:
df_modified.head()

A   B   C   D
0  64  64  34   1
1  64  97  72  61
2   9  29  71  10
3  57  46  66  59
4  10  25  10  28

In [14]:
%time df['K'] = df['A'].apply(lambda x: x*2)

CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 21.2 ms


Applying the function to chunks took much more time! 

_Reason_: `multiprocessing` has to serialize the data, send the serialized data to each thread, then apply the transformation, then send the data back to main

In [15]:
import multiprocessing
import time

In [16]:
def serial_func(arg1):
    return arg1*2

In [17]:
%time list(map(serial_func,[3,4,5]))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 49.6 µs


[6, 8, 10]

In [18]:
if __name__ == '__main__':
    start_time=time.time()
    res_list=[]
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:   # start worker processes
        res_list = pool.map(serial_func,[3,4,5]) # see https://docs.python.org/3/library/multiprocessing.html

    print('main elapsed:',time.time()-start_time,'seconds')

main elapsed: 0.3003065586090088 seconds


In [19]:
res_list

[6, 8, 10]